In [5]:
import os

def contar_imagenes(ruta):
    for categoria in os.listdir(ruta):
        path = os.path.join(ruta, categoria)
        count = len([nombre for nombre in os.listdir(path) if os.path.isfile(os.path.join(path, nombre))])
        print(f'{categoria}: {count}')

ruta_train = 'data_split/train'
ruta_val = 'data_split/val'
ruta_test = 'data_split/test'

print("Entrenamiento:")
contar_imagenes(ruta_train)

print("\nValidación:")
contar_imagenes(ruta_val)

print("\nPrueba:")
contar_imagenes(ruta_test)

Entrenamiento:
HSIL: 1089
LSIL: 870
Negative: 4338
ASC-H: 592
ASC-US: 387
SCC: 102

Validación:
HSIL: 273
LSIL: 218
Negative: 1085
ASC-H: 148
ASC-US: 97
SCC: 26

Prueba:
HSIL: 341
LSIL: 272
Negative: 1356
ASC-H: 185
ASC-US: 122
SCC: 33


In [4]:
import os
import numpy as np
from random import sample
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Directorios de datos
train_dir = 'data_split/train'
val_dir = 'data_split/val'
test_dir = 'data_split/test'

# Configuración de balanceo
target_train_count = 800  # Cantidad objetivo para el conjunto de entrenamiento
target_val_count = 200    # Cantidad objetivo para el conjunto de validación
target_test_count = 300   # Cantidad objetivo para el conjunto de prueba

# Función para balancear clases
def balance_classes(data_dir, target_count, datagen):
    classes = os.listdir(data_dir)
    
    for cls in classes:
        class_dir = os.path.join(data_dir, cls)
        images = os.listdir(class_dir)
        num_images = len(images)
        
        # Undersampling si la clase tiene más muestras que el objetivo
        if num_images > target_count:
            images_to_remove = sample(images, num_images - target_count)
            for img in images_to_remove:
                os.remove(os.path.join(class_dir, img))
            print(f"Reducción de {cls} a {target_count} imágenes por undersampling.")
        
        # Oversampling si la clase tiene menos muestras que el objetivo
        elif num_images < target_count:
            images_to_add = target_count - num_images
            while images_to_add > 0:
                img = sample(images, 1)[0]
                img_data = np.expand_dims(plt.imread(os.path.join(class_dir, img)), 0)
                aug_iter = datagen.flow(img_data, batch_size=1)
                aug_img = next(aug_iter)[0].astype(np.uint8)
                
                new_img_name = f"aug_{images_to_add}_{os.path.basename(img)}"
                new_img_path = os.path.join(class_dir, new_img_name)
                plt.imsave(new_img_path, aug_img)
                
                images_to_add -= 1
            print(f"Aumento de {cls} a {target_count} imágenes por oversampling.")

# Instancia del generador de aumento de datos
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)

# Balancear conjunto de entrenamiento
print("Balanceando conjunto de entrenamiento...")
balance_classes(train_dir, target_train_count, datagen)

# Balancear conjunto de validación
print("\nBalanceando conjunto de validación...")
balance_classes(val_dir, target_val_count, datagen)

# Balancear conjunto de prueba
print("\nBalanceando conjunto de prueba...")
balance_classes(test_dir, target_test_count, datagen)

# Verificación de los resultados
def count_images_in_directory(directory):
    return {cls: len(os.listdir(os.path.join(directory, cls))) for cls in os.listdir(directory)}

print("\nDistribución de imágenes después del balanceo:")
print("Entrenamiento:", count_images_in_directory(train_dir))
print("Validación:", count_images_in_directory(val_dir))
print("Prueba:", count_images_in_directory(test_dir))

Balanceando conjunto de entrenamiento...
Reducción de HSIL a 800 imágenes por undersampling.
Reducción de LSIL a 800 imágenes por undersampling.
Reducción de Negative a 800 imágenes por undersampling.
Aumento de ASC-H a 800 imágenes por oversampling.
Aumento de ASC-US a 800 imágenes por oversampling.
Aumento de SCC a 800 imágenes por oversampling.

Balanceando conjunto de validación...
Reducción de HSIL a 200 imágenes por undersampling.
Reducción de LSIL a 200 imágenes por undersampling.
Reducción de Negative a 200 imágenes por undersampling.
Aumento de ASC-H a 200 imágenes por oversampling.
Aumento de ASC-US a 200 imágenes por oversampling.
Aumento de SCC a 200 imágenes por oversampling.

Balanceando conjunto de prueba...
Reducción de HSIL a 300 imágenes por undersampling.
Aumento de LSIL a 300 imágenes por oversampling.
Reducción de Negative a 300 imágenes por undersampling.
Aumento de ASC-H a 300 imágenes por oversampling.
Aumento de ASC-US a 300 imágenes por oversampling.
Aumento d